In [ ]:
# VerifyPay MVP Prototype - Receipt Verifier 🚀

In [ ]:
# Install necessary packages
!pip install pytesseract opencv-python gradio openai
!apt-get install tesseract-ocr -y

In [ ]:
# Imports
import pytesseract
import cv2
import gradio as gr
import re
import hashlib
import openai
from datetime import datetime
from IPython.display import HTML

In [ ]:
# Set your OpenAI API Key here
openai.api_key = "YOUR_OPENAI_API_KEY"

In [ ]:
# OCR Function
def extract_text_from_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

In [ ]:
# Parse receipt fields
def parse_receipt(text):
    amount = re.search(r'Amount: ₦?(\d+[.,]?\d*)', text)
    sender = re.search(r'Sender: (.*)', text)
    receiver = re.search(r'Receiver: (.*)', text)
    date = re.search(r'Date: (.*)', text)
    
    return {
        'amount': amount.group(1) if amount else 'Not Found',
        'sender': sender.group(1) if sender else 'Not Found',
        'receiver': receiver.group(1) if receiver else 'Not Found',
        'date': date.group(1) if date else 'Not Found'
    }

In [ ]:
# Simulated blockchain hash registry
BLOCKCHAIN_DB = {
    hashlib.sha256(b'2025-04-12-Amount:5000000').hexdigest(): {
        "status": "registered",
        "registered_on": "2025-04-13 10:45:00"
    }
}

In [ ]:
def verify_transaction(parsed_data):
    key = f"{parsed_data['date']}-Amount:{parsed_data['amount']}"
    hashed_key = hashlib.sha256(key.encode()).hexdigest()
    return BLOCKCHAIN_DB.get(hashed_key, {"status": "not found"})

In [ ]:
# OpenAI LLM summary
def classify_with_llm(parsed_data):
    prompt = f"Summarize and classify this receipt:
{parsed_data}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [ ]:
# Generate HTML report
def generate_html_report(parsed, status, llm_feedback):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    html = f'''
    <html>
    <head><title>VerifyPay Report</title></head>
    <body style="font-family:sans-serif;">
        <h2>🧾 VerifyPay Receipt Report</h2>
        <p><strong>Generated:</strong> {now}</p>
        <hr>
        <h3>Parsed Fields:</h3>
        <ul>
            <li><strong>Amount:</strong> ₦{parsed['amount']}</li>
            <li><strong>Sender:</strong> {parsed['sender']}</li>
            <li><strong>Receiver:</strong> {parsed['receiver']}</li>
            <li><strong>Date:</strong> {parsed['date']}</li>
        </ul>
        <h3>Blockchain Status:</h3>
        <p>{status['status'].capitalize()}</p>
        <h3>LLM Feedback:</h3>
        <pre>{llm_feedback}</pre>
    </body>
    </html>
    '''
    report_path = "/mnt/data/receipt_report.html"
    with open(report_path, "w") as f:
        f.write(html)
    return report_path

In [ ]:
# Gradio UI
def receipt_verification_ui(image):
    text = extract_text_from_image(image)
    parsed = parse_receipt(text)
    blockchain_status = verify_transaction(parsed)
    llm_feedback = classify_with_llm(parsed)
    report_path = generate_html_report(parsed, blockchain_status, llm_feedback)
    
    result_msg = "✅ Verified Transaction" if blockchain_status['status'] == 'registered' else "❗Suspicious or Unverified"
    
    return parsed, result_msg, llm_feedback, report_path

In [ ]:
iface = gr.Interface(
    fn=receipt_verification_ui,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.JSON(label="Parsed Receipt Fields"),
        gr.Textbox(label="Verification Result"),
        gr.Textbox(label="LLM Classification Feedback"),
        gr.File(label="HTML Report Download")
    ],
    title="VerifyPay - Receipt Verifier with LLM & Blockchain Check",
    description="Upload a receipt to verify its authenticity, get AI feedback, and generate an HTML report."
)

In [ ]:
iface.launch()